In [3]:
import sqlalchemy
import pandas as pd
import numpy as np
from polygon import RESTClient
import datetime as dt
import os
from dotenv import load_dotenv
from connect import engine, Base, Company_Financials
from sqlalchemy import select, inspect
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql import func
import logging
from sqlalchemy.dialects.postgresql import JSONB
# from log_config import setup_logging
import pytz
import json

In [3]:
setup_logging()

In [4]:
load_dotenv()
key = os.getenv("API_KEY")

In [5]:
client = RESTClient(api_key=key)

In [6]:
events = client.get_ticker_events("META")
print(events)

TickerChangeResults(name='Meta Platforms, Inc. Class A Common Stock', composite_figi='BBG000MM2P62', cik='0001326801', events=[{'ticker_change': {'ticker': 'META'}, 'type': 'ticker_change', 'date': '2022-06-09'}, {'ticker_change': {'ticker': 'FB'}, 'type': 'ticker_change', 'date': '2012-05-18'}])


In [9]:
events

TickerChangeResults(name='Meta Platforms, Inc. Class A Common Stock', composite_figi='BBG000MM2P62', cik='0001326801', events=[{'ticker_change': {'ticker': 'META'}, 'type': 'ticker_change', 'date': '2022-06-09'}, {'ticker_change': {'ticker': 'FB'}, 'type': 'ticker_change', 'date': '2012-05-18'}])

In [6]:
wiki = 'http://en.wikipedia.org/wiki'
djia_ticker_list = wiki + '/Dow_Jones_Industrial_Average'
sp500_tickers_list = wiki + '/List_of_S%26P_500_companies'
tickersSP500 = pd.read_html(sp500_tickers_list)[0].Symbol.to_list()
djia_tickers = pd.read_html(djia_ticker_list)[1].Symbol.to_list()

In [48]:
class Company_Financials_updater:
    def __init__(self, tickers, engine):
        self.tickers = tickers if isinstance(tickers, list) else [tickers]
        self.engine = engine
        
    def transform_data(self, df):
        df['start_date'] = pd.to_datetime(df['start_date'],errors='coerce')
        df['end_date'] = pd.to_datetime(df['end_date'],errors='coerce')
        df['filing_date'] = pd.to_datetime(df['filing_date'],errors='coerce')
        df['filing_date'] = df['filing_date'].dt.strftime('%Y-%m-%d %H:%M:%S') if df['filing_date'] is not pd.NaT else None
        df.drop(columns=['cik', 'source_filing_file_url', 'source_filing_url'], inplace=True)
        df['financials'] = df['financials'].apply(json.dumps)
        transformed = df.to_dict(orient='records')
        return transformed
    
    def need_for_update(self):
        pass
        
    def update_data(self,client):
        with self.engine.connect() as conn:
            all_data = []
            logging.info(f"Updating company financials for {self.tickers}")
            for ticker in self.tickers:
                resp = client.vx.list_stock_financials(ticker)
                ticker_df = pd.DataFrame(resp)
                
                if not ticker_df.empty:
                    transformed_data = self.transform_data(ticker_df)
                    all_data.extend(transformed_data)
                else:
                    print(f"No data for {ticker}")
        return all_data        
                    
            # if all_data:
            #     try:
            #         conn.execute(Company_Financials.__table__.insert(), all_data)
            #         logging.info("Data insert completed successfully.")

            #     except Exception as e:
            #             logging.error(f"Error updating company data for {ticker}: {e}")
                        
    

In [49]:
updater =  Company_Financials_updater(tickers = 'AAPL', engine = engine)


In [50]:
t = updater.update_data(client)

In [51]:
t

[{'company_name': 'APPLE INC',
  'end_date': Timestamp('2023-12-30 00:00:00'),
  'filing_date': nan,
  'financials': '{"balance_sheet": {"inventory": {"formula": null, "label": "Inventory", "order": 230, "unit": "USD", "value": 6511000000.0, "xpath": null}, "fixed_assets": {"formula": null, "label": "Fixed Assets", "order": 320, "unit": "USD", "value": 43666000000.0, "xpath": null}, "current_assets": {"formula": null, "label": "Current Assets", "order": 200, "unit": "USD", "value": 143692000000.0, "xpath": null}, "other_noncurrent_liabilities": {"formula": null, "label": "Other Non-current Liabilities", "order": 820, "unit": "USD", "value": 39441000000.0, "xpath": null}, "liabilities": {"formula": null, "label": "Liabilities", "order": 600, "unit": "USD", "value": 279414000000.0, "xpath": null}, "other_noncurrent_assets": {"formula": null, "label": "Other Non-current Assets", "order": 350, "unit": "USD", "value": 166156000000.0, "xpath": null}, "long_term_debt": {"formula": null, "labe